## Evapotranspiration Interpolation, Accuracy Assessment and PostGIS 

### Data Preparation
- Load ET data for July 2023 and September 2023.
- Prepare ground-truth data.

### Interpolation
- Perform IDW, Ordinary Kriging, and Universal Kriging interpolation.

### Accuracy Assessment
- Compare interpolated values with ground-truth data.
- Calculate RMSE, MAE, and R² for each method.

### Sending best model to Postgis
  - We will send best fit models to our remote databas
#### Link to datasets:  https://app.climateengine.org/climateEngine



### import requires packages

In [2]:
import arcpy
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math
from math import sqrt
import random
import psycopg2

### ET July 2023  extract by mask for corn areas
#### Since we are working with corn growers we want to provide the et in the areas were corn are grown in 2023. We used Mn Land cover data for 2023 corn areas and Clipped.tiff. 

In [11]:
#July 2023 
arcpy.env.scratchWorkspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

with arcpy.EnvManager(scratchWorkspace=arcpy.env.scratchWorkspace):
    ETCorn = arcpy.sa.ExtractByMask(
        in_raster="ET_Jul2023.aet.tif",
        in_mask_data="clipped.TIF",
        extraction_area="INSIDE",
        analysis_extent='-97.3578140352203 43.3388462071335 -88.9958333 49.5041666666667 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
    )
    ETCorn.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExtractETCorn")
#September 2023
arcpy.env.scratchWorkspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

with arcpy.EnvManager(scratchWorkspace=arcpy.env.scratchWorkspace):
    ETCornSep = arcpy.sa.ExtractByMask(
        in_raster="ET_Sep2023.aet.tif",
        in_mask_data="clipped.TIF",
        extraction_area="INSIDE",
        analysis_extent='-97.3578140352203 43.3388462071335 -88.9958333 49.5041666666667 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
    )
    ETCornSep.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExtractETCornSep")


### Raster to Point
#### Once we have the raster data for the corn areas, we converted to points and these pointsare our ground truth data

In [12]:
#July 2023
arcpy.conversion.RasterToPoint(
    in_raster="ETCorn",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\RetToP",
    raster_field="Value"
)
#September 2023
arcpy.conversion.RasterToPoint(
    in_raster="ETCornSEP",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\RetToPSEP",
    raster_field="Value"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\RetToPSEP'>

### ET interpolated data

#### Once we have the points we use 3 models to interpolate these points : IDW, IDW ordinary krigging and IDW universal krigging

In [17]:
#July 2023
# IDW
ET_idw = "ET2023_IDW"

# ORDINARY_KRIGING
ET_ord_kri = "ET2023_ord_krigging"

# UNIVERSAL_KRIGING
ET_uni_kri = "ET2023_uni_krigging"

#September 2023
# IDW
ETSep_idw = "ETSep2023_Idw"

# ORDINARY_KRIGING
ETSep_ord_kri = "ETSEP2023_ord_krigging"

# UNIVERSAL_KRIGING
ETSep_uni_kri = "ETSEP2023_uni_krigging"

### Interpolated raster to points
#### Again we convert all three interpolated rasters to point to get point data sets and compare those with out ground truth data. 

In [18]:
#July 2023
import arcpy

# Set the workspace where the output feature classes will be saved
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

# IDW
arcpy.conversion.RasterToPoint(in_raster=ET_idw, out_point_features="ET2023_IDWpoints", raster_field="Value")

# ORDINARY_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ET_ord_kri, out_point_features="ET2023_ord_Kriggingpoints", raster_field="Value")

# UNIVERSAL_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ET_uni_kri, out_point_features="ET2023_uni_Kriggingpoints", raster_field="Value")
# September 2023
# Set the workspace where the output feature classes will be saved
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

# IDW
arcpy.conversion.RasterToPoint(in_raster=ETSep_idw, out_point_features="ETSep2023_Idwpoints", raster_field="Value")

# ORDINARY_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ETSep_ord_kri, out_point_features="ETSep2023_ord_Kriggingpoints", raster_field="Value")

# UNIVERSAL_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ETSep_uni_kri, out_point_features="ETSep2023_uni_Kriggingpoints", raster_field="Value")

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\ETSep2023_uni_Kriggingpoints'>

####  Checking Exploratory Interpolation

In [1]:
# July 2023 
arcpy.ga.ExploratoryInterpolation(
    in_features="RetToP",
    value_field="pointid",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExploratoryInterpolation",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

#September 2023
arcpy.ga.ExploratoryInterpolation(
    in_features="RetToPSEP",
    value_field="pointid",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExploratoryInterpolationSep2023",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)


<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\ExploratoryInterpolationSep2023'>

In [21]:

# Set  workspace (where your data is stored)
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

# Define paths to datasets July 2023
interpolated_layer_IDW = "ET2023_IDWpoints"
interpolated_layer_ord_Krigging = "ET2023_ord_Kriggingpoints"
interpolated_layer_uni_Krigging = "ET2023_uni_Kriggingpoints"
truth_layer = "RetToP"

# Define paths to datasets September 2023
interpolated_layer_IDWSep = "ETSep2023_Idwpoints"
interpolated_layer_ord_KriggingSep = "ETSep2023_ord_Kriggingpoints"
interpolated_layer_uni_KriggingSep = "ETSep2023_uni_Kriggingpoints"
truth_layer = "RetToPSep"

### Accuracy assessment 

#### Once we have points from interpolated datasets, we do accuracy assessment of spatial interpolation methods by comparing interpolated data against truth data. It identifies closest matches, randomly samples 50% of these, and calculates statistical metrics like RMSE, MAE, and R² to assess methods such as IDW, Ordinary Krigging, and Universal Krigging.

In [ ]:
# July 2023 

def process_interpolation(interpolated_layer, truth_layer, label):
    # Read the points into memory
    interpolated_points = [row for row in arcpy.da.SearchCursor(interpolated_layer, ["SHAPE@", "grid_code"])]
    truth_points = [row for row in arcpy.da.SearchCursor(truth_layer, ["SHAPE@", "grid_code"])]

    # Function to find closest point
    def find_closest(inter_point, truth_points):
        min_distance = float('inf')
        closest_truth = None
        for truth_point in truth_points:
            dist = inter_point[0].distanceTo(truth_point[0])
            if dist < min_distance:
                min_distance = dist
                closest_truth = truth_point
        return (inter_point[1], closest_truth[1], min_distance)

    # Find closest matches
    closest_matches = [find_closest(inter_point, truth_points) for inter_point in interpolated_points]

    # Sample 50% of the data randomly
    sampled_matches = random.sample(closest_matches, len(closest_matches) // 2)

    # Function to calculate RMSE, MAE, and R²
    def calculate_metrics(data):
        true_values, predicted_values = zip(*[(true, pred) for true, pred, _ in data])
        mse = sum((t - p) ** 2 for t, p in zip(true_values, predicted_values)) / len(data)
        mae = sum(abs(t - p) for t, p in zip(true_values, predicted_values)) / len(data)
        rmse = sqrt(mse)
        mean_true = sum(true_values) / len(data)
        ss_tot = sum((t - mean_true) ** 2 for t in true_values)
        ss_res = sum((t - p) ** 2 for t, p in zip(true_values, predicted_values))
        r_squared = 1 - (ss_res / ss_tot)
        return rmse, mae, r_squared

    # Calculate and print the accuracy metrics
    rmse, mae, r_squared = calculate_metrics(sampled_matches)
    print(f"{label} RMSE: {rmse}, MAE: {mae}, R²: {r_squared}")

# Define layers
truth_layer = "RetToP"
interpolated_layer_IDW = "ET2023_IDWpoints"
interpolated_layer_ord_Krigging = "ET2023_ord_Kriggingpoints"
interpolated_layer_uni_Krigging = "ET2023_uni_Kriggingpoints"

# Process each interpolation method
process_interpolation(interpolated_layer_IDW, truth_layer, "IDW")
process_interpolation(interpolated_layer_ord_Krigging, truth_layer, "Ordinary Krigging")
process_interpolation(interpolated_layer_uni_Krigging, truth_layer, "Universal Krigging")


IDW RMSE: 1.3469024620656782, MAE: 0.7330507175877402, R²: 0.9915695020869919


In [22]:
# September 2023
def process_interpolation(interpolated_layerSep, truth_layerSep, labelSep):
    # Read the points into memory
    interpolated_pointsSep = [row for row in arcpy.da.SearchCursor(interpolated_layerSep, ["SHAPE@", "grid_code"])]
    truth_pointsSep = [row for row in arcpy.da.SearchCursor(truth_layerSep, ["SHAPE@", "grid_code"])]

    # Function to find closest point
    def find_closest(inter_pointSep, truth_pointsSep):
        min_distance = float('inf')
        closest_truthSep = None
        for truth_pointSep in truth_pointsSep:
            dist = inter_pointSep[0].distanceTo(truth_pointSep[0])
            if dist < min_distance:
                min_distance = dist
                closest_truthSep = truth_pointSep
        return (inter_pointSep[1], closest_truthSep[1], min_distance)

    # Find closest matches
    closest_matchesSep = [find_closest(inter_pointSep, truth_pointsSep) for inter_pointSep in interpolated_pointsSep]

    # Sample 50% of the data randomly
    sampled_matchesSep = random.sample(closest_matchesSep, len(closest_matchesSep) // 2)

    # Function to calculate RMSE, MAE, and R²
    def calculate_metrics(dataSep):
        true_valuesSep, predicted_valuesSep = zip(*[(trueSep, predSep) for trueSep, predSep, _ in dataSep])
        mse = sum((t - p) ** 2 for t, p in zip(true_valuesSep, predicted_valuesSep)) / len(dataSep)
        mae = sum(abs(t - p) for t, p in zip(true_valuesSep, predicted_valuesSep)) / len(dataSep)
        rmse = sqrt(mse)
        mean_trueSep = sum(true_valuesSep) / len(dataSep)
        ss_tot = sum((t - mean_trueSep) ** 2 for t in true_valuesSep)
        ss_res = sum((t - p) ** 2 for t, p in zip(true_valuesSep, predicted_valuesSep))
        r_squared = 1 - (ss_res / ss_tot)
        return rmse, mae, r_squared

    # Calculate and print the accuracy metrics
    rmseSep, maeSep, r_squaredSep = calculate_metrics(sampled_matchesSep)
    print(f"{labelSep} RMSE: {rmseSep}, MAE: {maeSep}, R²: {r_squaredSep}")

# Define layers for September
truth_layerSep = "RetToPSep"
interpolated_layer_IDWSep = "ETSep2023_Idwpoints"
interpolated_layer_ord_KriggingSep = "ETSep2023_ord_Kriggingpoints"
interpolated_layer_uni_KriggingSep = "ETSep2023_uni_Kriggingpoints"

# Process each interpolation method for September
process_interpolation(interpolated_layer_IDWSep, truth_layerSep, "IDWSep")
process_interpolation(interpolated_layer_ord_KriggingSep, truth_layerSep, "Ordinary KriggingSep")
process_interpolation(interpolated_layer_uni_KriggingSep, truth_layerSep, "Universal KriggingSep")


IDWSep RMSE: 0.9680784017054407, MAE: 0.6273982147000878, R²: 0.9967054012233997
Ordinary KriggingSep RMSE: 1.0453052820712923, MAE: 0.7249511565802235, R²: 0.9962227417907644
Universal KriggingSep RMSE: 1.2638997082992183, MAE: 0.9147894954681393, R²: 0.9943041283507762


### Choosing the Best Model and sending it to remore database

#### IDW Model for July 2023 seems like a best fit amongst others we are using IDW for our map and sending this to postgis.  Although all the model seems pretty good but I choose IDW out of them because it has slightly less rmse and mae and r-2 of 0.99

#### We will also choose IDW for September 2023  because it looks better than others with IDWSep RMSE: 0.9802416925892986, MAE: 0.6344020543008491, R²: 0.996608592848134


In [27]:
# Function to convert raster to point and push to database
def point_and_push(raster_name, local_gdb_path, geodatabase_path):
    # Convert raster to point data
    arcpy.conversion.RasterToPoint(
        in_raster=os.path.join(local_gdb_path, raster_name),
        out_point_features=os.path.join(local_gdb_path, raster_name + '_point'),
        raster_field="Value"
    )
    print(f"{raster_name} converted to point")

    # Push point data to remote database
    arcpy.conversion.FeatureClassToGeodatabase(
        os.path.join(local_gdb_path, raster_name + '_point'),
        geodatabase_path
    )
    print(f"{raster_name}_point pushed to database")

# Paths
local_gdb_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"
geodatabase_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Interpolation raster names
interpolation_rasters = [
    "ET2023_IDW", 
"ETSep2023_Idw"]

# Convert each raster to point and push to the database
for raster_name in interpolation_rasters:
    point_and_push(raster_name, local_gdb_path, geodatabase_path)


ET2023_IDW converted to point
ET2023_IDW_point pushed to database
ETSep2023_Idw converted to point
ETSep2023_Idw_point pushed to database
